In [2]:
import os
import re
import string

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

'''from nltk.tokenize import WordPunctTokenizer
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
'''
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
yelp_business = pd.read_json('../input/yelp_academic_dataset_business.json', lines=True)
yelp_business.fillna('NA', inplace=True)
# we want to make sure we only work with restaurants -- nothing else
restaurants = yelp_business[yelp_business['categories'].str.contains('Restaurants')]
print('Number of all businesses: ',yelp_business.shape[0])
print(f"Shape of restaurants dataset{restaurants.shape}")

Number of all businesses:  192609
Shape of restaurants dataset(59371, 14)


In [4]:
yelp_business.head()

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,postal_code,review_count,stars,state
0,2818 E Camino Acequia Drive,{'GoodForKids': 'False'},1SWheh84yJXfytovILXOAQ,"Golf, Active Life",Phoenix,NA,0,33.522143,-112.018481,Arizona Biltmore Golf Club,85016,5,3.0,AZ
1,30 Eglinton Avenue W,"{'RestaurantsReservations': 'True', 'GoodForMe...",QXAEGFB4oINsVuTFxEYKFQ,"Specialty Food, Restaurants, Dim Sum, Imported...",Mississauga,"{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W...",1,43.605499,-79.652289,Emerald Chinese Restaurant,L5R 3E7,128,2.5,ON
2,"10110 Johnston Rd, Ste 15","{'GoodForKids': 'True', 'NoiseLevel': 'u'avera...",gnKjwL_1w79qoiV3IC_xQQ,"Sushi Bars, Restaurants, Japanese",Charlotte,"{'Monday': '17:30-21:30', 'Wednesday': '17:30-...",1,35.092564,-80.859132,Musashi Japanese Restaurant,28210,170,4.0,NC
3,"15655 W Roosevelt St, Ste 237",NA,xvX2CttrVhyG2z1dFg_0xw,"Insurance, Financial Services",Goodyear,"{'Monday': '8:0-17:0', 'Tuesday': '8:0-17:0', ...",1,33.455613,-112.395596,Farmers Insurance - Paul Lorenz,85338,3,5.0,AZ
4,"4209 Stuart Andrew Blvd, Ste F","{'BusinessAcceptsBitcoin': 'False', 'ByAppoint...",HhyxOkGAM07SRYtlQ4wMFQ,"Plumbing, Shopping, Local Services, Home Servi...",Charlotte,"{'Monday': '7:0-23:0', 'Tuesday': '7:0-23:0', ...",1,35.190012,-80.887223,Queen City Plumbing,28217,4,4.0,NC


In [5]:
restaurants.head()

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,postal_code,review_count,stars,state
1,30 Eglinton Avenue W,"{'RestaurantsReservations': 'True', 'GoodForMe...",QXAEGFB4oINsVuTFxEYKFQ,"Specialty Food, Restaurants, Dim Sum, Imported...",Mississauga,"{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W...",1,43.605499,-79.652289,Emerald Chinese Restaurant,L5R 3E7,128,2.5,ON
2,"10110 Johnston Rd, Ste 15","{'GoodForKids': 'True', 'NoiseLevel': 'u'avera...",gnKjwL_1w79qoiV3IC_xQQ,"Sushi Bars, Restaurants, Japanese",Charlotte,"{'Monday': '17:30-21:30', 'Wednesday': '17:30-...",1,35.092564,-80.859132,Musashi Japanese Restaurant,28210,170,4.0,NC
11,2450 E Indian School Rd,"{'RestaurantsTakeOut': 'True', 'BusinessParkin...",1Dfx3zM-rW4n-31KeC8sJg,"Restaurants, Breakfast & Brunch, Mexican, Taco...",Phoenix,"{'Monday': '7:0-0:0', 'Tuesday': '7:0-0:0', 'W...",1,33.495194,-112.028588,Taco Bell,85016,18,3.0,AZ
13,5981 Andrews Rd,"{'RestaurantsPriceRange2': '2', 'BusinessAccep...",fweCYi8FmbJXHCqLnwuk8w,"Italian, Restaurants, Pizza, Chicken Wings",Mentor-on-the-Lake,"{'Monday': '10:0-0:0', 'Tuesday': '10:0-0:0', ...",1,41.708520,-81.359556,Marco's Pizza,44060,16,4.0,OH
17,"1775 E Tropicana Ave, Ste 29","{'OutdoorSeating': 'False', 'BusinessAcceptsCr...",PZ-LZzSlhSe9utkQYU8pFg,"Restaurants, Italian",Las Vegas,NA,0,36.100016,-115.128529,Carluccio's Tivoli Gardens,89119,40,4.0,NV


Now we bring the reviews and perform some preprocessing on those reviews..

In [6]:
yelp_review_iter = pd.read_json('../input/yelp_academic_dataset_review.json', chunksize=100000, lines=True)

Because reviews are too big, we will read them in chunks, and make sure we delete reviews of places that are not in our list of businesses filtered earlier. Note here we choose 5 chunks, but we could have chosen any number (larger numbers will give MemoryError later on).

In [7]:
yelp_review = pd.DataFrame()
i=0
for df in yelp_review_iter:
    
    df = df[df['business_id'].isin(restaurants['business_id'])]
    print(df.shape)
    yelp_review = pd.concat([yelp_review, df])
    print(i)
    if i==70: break
    i=i+1

(63667, 9)
0
(63513, 9)
1
(63553, 9)
2
(63438, 9)
3
(63436, 9)
4
(61381, 9)
5
(61385, 9)
6
(64316, 9)
7
(64440, 9)
8
(64463, 9)
9
(64385, 9)
10
(64350, 9)
11
(60315, 9)
12
(61715, 9)
13
(62325, 9)
14
(62100, 9)
15
(62006, 9)
16
(62450, 9)
17
(62260, 9)
18
(58197, 9)
19
(64582, 9)
20
(64677, 9)
21
(64436, 9)
22
(64955, 9)
23
(65230, 9)
24
(62838, 9)
25
(59724, 9)
26
(62860, 9)
27
(62225, 9)
28
(62694, 9)
29
(62827, 9)
30
(62418, 9)
31
(58841, 9)
32
(61403, 9)
33
(62699, 9)
34
(62716, 9)
35
(62656, 9)
36
(62866, 9)
37
(62164, 9)
38
(58083, 9)
39
(63879, 9)
40
(63876, 9)
41
(63709, 9)
42
(63954, 9)
43
(64148, 9)
44
(61538, 9)
45
(61399, 9)
46
(65045, 9)
47
(64972, 9)
48
(64830, 9)
49
(64840, 9)
50
(65075, 9)
51
(61621, 9)
52
(62867, 9)
53
(64094, 9)
54
(64100, 9)
55
(64137, 9)
56
(64189, 9)
57
(64009, 9)
58
(60031, 9)
59
(62307, 9)
60
(62883, 9)
61
(62378, 9)
62
(62489, 9)
63
(63068, 9)
64
(61754, 9)
65
(49903, 9)
66


In [8]:
yelp_review.shape

(4201684, 9)

Also make sure we only get businesses that already show up in our review list and delete the rest.

In [9]:
import pickle
yelp_review.to_pickle("pickled_reviews.pickle")


In [10]:
rest_reviews =pd.read_pickle("pickled_reviews.pickle")

In [11]:
yelp_business = yelp_business[yelp_business['business_id'].isin(rest_reviews['business_id'])]

In [12]:
print('Final businesses shape: ', yelp_business.shape)
print('Final review shape: ', rest_reviews.shape)

Final businesses shape:  (59371, 14)
Final review shape:  (4201684, 9)


In [13]:
rest_reviews.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
3,ikCg8xy5JIg_NGPx-MSIDA,0,2018-01-09 20:56:38,0,yi0R0Ugj_xUx_Nek0-_Qig,5,Went in for a lunch. Steak sandwich was delici...,0,dacAIZ6fTM6mqwW5uxkskg
5,eU_713ec6fTGNO4BegRaww,0,2013-01-20 13:25:59,0,fdiNeiN_hoCxCMy2wTRW9g,4,I'll be the first to admit that I was not exci...,0,w31MKYsNFMrjhWxxAb5wIw
6,3fw2X5bZYeW9xCz_zGhOHg,5,2016-05-07 01:21:02,4,G7XHMxG0bx9oBJNECG4IFg,3,Tracy dessert had a big name in Hong Kong and ...,5,jlu4CztcSxrKx56ba1a5AQ
7,zvO-PJCpNk4fgAVUnExYAA,1,2010-10-05 19:12:35,1,8e9HxxLjjqc9ez5ezzN7iQ,1,This place has gone down hill. Clearly they h...,3,d6xvYpyzcfbF_AZ8vMB7QA
10,8mIrX_LrOnAqWsB5JrOojQ,0,2011-11-30 02:11:15,0,kbtscdyz6lvrtGjD1quQTg,4,"Like walking back in time, every Saturday morn...",0,FIk4lQQu1eTe2EpzQ4xhBA


In [14]:
rest_reviews['funny'].describe()

count    4.201684e+06
mean     4.553600e-01
std      2.460489e+00
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      1.290000e+03
Name: funny, dtype: float64

In [15]:
pd.options.display.max_seq_items = 2000
print(rest_reviews[rest_reviews['funny']==1290][['business_id', 'text']])

                    business_id                                               text
1331304  DN0b4Un8--Uf6SEWLeh0UA  Flew to Arizona a few months ago to try this. ...


In [16]:
# Check:
print( (rest_reviews['funny']>4).mean())
print(f"Number of funny reviews:{(rest_reviews['funny']>4).sum()}")
#print(rest_reviews['fun_bin'].mean())

0.017914007812105812
Number of funny reviews:75269


In [17]:
75269/4201684

0.017914007812105812

In [18]:
rest_reviews['fun_bin']=rest_reviews['funny'].apply(lambda x: 1 if x>4 else 0)

In [19]:
print(rest_reviews['fun_bin'].mean())

0.017914007812105812


#### Getting a df with funny reviews

In [20]:
rest_reviews_fun = rest_reviews[rest_reviews['fun_bin']==1]

In [21]:
rest_reviews_fun.shape

(75269, 10)

In [22]:
rest_reviews_fun.drop_duplicates(subset= 'text', inplace=True)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [23]:
rest_reviews_fun.shape

(75251, 10)

### Sampling not funny reviews


In [24]:
rest_reviews_not_fun = rest_reviews[rest_reviews['fun_bin']==0]

In [25]:
idx = rest_reviews_not_fun.index.values

In [26]:
len(idx)

4126415

In [27]:
# Balancing the classes: getting the same number of not funny reviews as funny
#random_hotels = np.random.choice(neg_activity_df["hotel"].unique(), len(neg_activity_df))
random_idx = np.random.choice(idx,rest_reviews_fun.shape[0])

In [28]:
len(random_idx)

75251

In [29]:
rest_reviews_not_fun = rest_reviews_not_fun.loc[random_idx,:].copy()

In [30]:
rest_reviews_not_fun.shape

(75251, 10)

In [31]:
reviews_final = pd.concat([rest_reviews_fun, rest_reviews_not_fun])

In [32]:
reviews_final.shape

(150502, 10)

In [33]:
reviews_final.to_csv("../balanced_reviews.csv")

In [34]:
reviews_final = pd.read_csv("../balanced_reviews.csv", index_col=0)

In [35]:
reviews_final.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id,fun_bin
17,cHdJXLlKNWixBXpDwEGb_A,1,2015-04-01 16:30:00,7,6BnQwlxRn7ZuWdzninM9sQ,3,I love chinese food and I love mexican food. W...,1,JSrP-dUmLlwZiI7Dp3PQ2A,1
21,Mem13A3C202RzT53npn4NA,9,2017-05-13 10:41:43,6,IPw8yWiyqnfBzzWmypUHgg,5,If you are looking for the best pierogies in P...,9,5JVY32_bmTBfIGpCCsnAfw,1
62,SU56w479vUfFHsvmvQIf7A,6,2016-07-25 03:55:20,5,E4LqIZ7DJd_R4ZHSNKx4RQ,4,So good! They didn't make it to 5 stars due to...,7,DoRCeCcJbrsM2BiAKj3trA,1
126,tjAeaGdxf7I4xN9M7wGJNQ,4,2014-07-13 14:32:56,5,TaoaX7MqCujFRNaJBns2Sw,5,While the prices are a bit high for a make-you...,8,x37OyP--VEFE5p-xreplYA,1
246,FhIeCF6QrsLaRvAeu0oEPQ,4,2013-06-24 06:42:29,5,3Qc49B7dA0ONmCxrn5iwCQ,2,OVERALL: The food isn't good (I explain below)...,13,2k8OVAPxlXHsA5X6EIoQpQ,1


In [36]:
df_reviews = reviews_final[['funny','text', 'fun_bin']]

In [37]:
pd.set_option('display.max_colwidth', -1)
df_reviews.head()

,funny,text,fun_bin
17,7,"I love chinese food and I love mexican food. What can go wrong? A couple of things. First things first, this place is more of a ""rice bowl"" kind of place. I thought it was going to be more diverse as far as the menu goes, but its mainly rice bowls you get with different kinds of meats. The ordering was a little confusing at first, but one of the employees helped us out and I got the 2-item bowl and got the jade chicken and hengrenade chicken with all rice(jerk). I also ordered a jade chicken quesadilla on the side.\n\nI'm gonna admit, this place looks kinda dirty. I don't think Arizona uses those health department letter grade system like California does, but if I were to just judge by how it looked inside, i'd give it a ""C"" grade lol. We waited for about 15 minutes or so and finally got our food. We took it to go and ate at our hotel room. \n\nMmmm... the food was just alright. The jade chicken was nothing special. It tasted like any generic chinese fast food orange chicken/sesame chicken variant. The hengrenade chicken, although was the less spicier version of the jerk chicken, was still pretty spicy for me. Just be warned the jerk chicken is super spicy. If you aren't sure, ask for a sample at the restaurant before ordering, but it was way too spicy for me. \n\nThe jade chicken quesadilla was decent, but nothing special. Just imagine orange chicken in between a tortilla and cheese. A friend of mine ordered a jade chicken burrito and we were confused when we pulled it out of the bag because it was literally the size of Mcdonald's apple pie. If you order the burrito, be warned that it's a burrito for gnomes and smurfs, but he said it was tasty. \n\nThey provide a snicker doodle sugar cookie for each meal and it was decent, again nothing special. \n\nNot gonna lie, the next day my stomach felt like a little mexican dude and chinese dude were wrestling and throwing molotov cocktails inside. I used the bathroom like 5 times. I don't recommend eating this place if you have a lot to do the next day.",1
21,6,"If you are looking for the best pierogies in Pittsburgh, this is your place. There are a few small tables outside but most of the business is carry out. Pierogies Plus wins Best Pierogies every year. Why? Because the owner is from Poland and she is making the real deal pierogies. The best part is that they are hand pinched by a group of older Polish and Hungarian women. \nThe biggest seller is potato and cheese but they sell many flavors. They are like plump pillows of softness. You can buy them buy the dozen. You can get them cold to take home and freeze or warm and ready to eat. The warm ones are served with butter and onions. It's definitely a comfort food. The best part is that they ship internationally. Yes, they are that good.",1
62,5,"So good! They didn't make it to 5 stars due to the prices are a bit high for the amount of food and the location is a bit unsavory. \nThe decor and atmosphere was surprisingly nice, from the outside I expected to be more run down inside. The staff was very nice. We were surprised how empty the dining room was for a Friday evening.\nWe got Vegetable Samosas to start then ordered Chicken Tikka Masala, Lamb Rogan Josh, rice and plain Naan. Our only complaint was the lamb could've been more tender but everything was flavorful and delicious. \nI would definitely go again if given the chance.",1
126,5,"While the prices are a bit high for a make-your-own pizza, the taste makes up for it. I love going to Seventh Street market, sitting Not Just Coffee and having a drink while waiting for delicious fresh made pizza from Pure. I've taken this to go as well as eaten inside the market, and I can say that the pizza doesn't do well reheated. So try to eat it fresh while there if possible.\n\nIf one of their specialty pizzas sounds good to you, go for it, as those are definitely a better deal for the amount of toppings you get for the money. I wanted what I wanted, though, so

### Data pre-processing

#### Goals
- Keep punctuation
- Split by ".", "!" to account for misspeling (like "Hi!I went to...")

In [38]:
!pip install spacymoji

In [39]:
import pandas as pd
import numpy as np
from string import punctuation
import os
import spacy
import string
import re
from spacy.symbols import ORTH
from collections import Counter

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence 
from spacymoji import Emoji

In [40]:

re_br = re.compile(r'<\s*br\s*/?>', re.IGNORECASE)
def sub_br(x): return re_br.sub("\n", x)

#nlp = spacy.load("en")
nlp = spacy.load('en_core_web_sm')
spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS

spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS

def clean_text(text):
    ''' Pre process and convert texts to a list of words '''
    text = str(text)
    text = text.lower()

    # Clean the text
   # text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=\(\)]", " ", text) # keep punctuatuin, numnbers and letters
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " . ", text) #Add space to the dot
    text = re.sub(r"!", " ! ", text) #Add space to the exclamation sign
    text = re.sub(r":", " :", text) #Add space before : sign
    text = re.sub(r";", " ;", text) #Add space before ; sign
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    #text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    #text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    # find emojis
    emoji_list = []
    '''
    for word in text.split():
        if any(char in emoji.UNICODE_EMOJI for char in word):
            emoji_list.append(word)
    emoji_list'''
    #text = text.split()

    return text

my_tok = spacy.load('en')
emoji = Emoji(my_tok)
my_tok.add_pipe(emoji, first=True)
def spacy_tok(x): return [tok.text for tok in my_tok.tokenizer(clean_text(x))]

def remove_stop_words(tokens): return [tok for tok in tokens if tok not in spacy_stopwords]

In [41]:
text = "I'm soooo excited!!!!!This is 10000% the best place on earth:))))) 😃..."

In [42]:
text

"I'm soooo excited!!!!!This is 10000% the best place on earth:))))) 😃..."

In [43]:
clean_text(text)

'i am soooo excited ! ! ! ! ! this is 10000% the best place on earth :))))) 😃 . . . '

In [44]:
spacy_tok(clean_text(text))

['i',
 'am',
 'soooo',
 'excited',
 '!',
 '!',
 '!',
 '!',
 '!',
 'this',
 'is',
 '10000',
 '%',
 'the',
 'best',
 'place',
 'on',
 'earth',
 ':',
 ')',
 ')',
 ')',
 ')',
 ')',
 '😃',
 '.',
 '.',
 '.']

In [45]:
text2 = "I also ordered a jade chicken quesadilla on the side.\n\nI'm gonna admit, this place looks kinda dirty. I don't think Arizona uses those health department letter grade system like California does, but if I were to just judge by how it looked inside, i'd give it a 'C' grade lol 😃"

In [46]:
remove_stop_words(spacy_tok(clean_text(text2)))

['ordered',
 'jade',
 'chicken',
 'quesadilla',
 '.',
 'gon',
 'na',
 'admit',
 'place',
 'looks',
 'kinda',
 'dirty',
 '.',
 'think',
 'arizona',
 'uses',
 'health',
 'department',
 'letter',
 'grade',
 'system',
 'like',
 'california',
 'judge',
 'looked',
 'inside',
 'c',
 'grade',
 'lol',
 '😃']

### Building a vocabulary

In [47]:
counts = Counter()
for sent in df_reviews['text']:
    try:
        counts.update(remove_stop_words(spacy_tok(sent)))
    except:
        pass

In [48]:
counts

Counter({'love': 31860,
         'chinese': 7307,
         'food': 132849,
         'mexican': 6940,
         '.': 1977613,
         'wrong': 8100,
         '?': 76948,
         'couple': 10281,
         'things': 14271,
         'place': 106364,
         '"': 129599,
         'rice': 20417,
         'bowl': 9045,
         'kind': 12409,
         'thought': 13775,
         'going': 24073,
         'diverse': 470,
         'far': 9373,
         'menu': 44722,
         'goes': 3717,
         'mainly': 754,
         'bowls': 1961,
         'different': 15021,
         'kinds': 1377,
         'meats': 3825,
         'ordering': 7140,
         'little': 36360,
         'confusing': 687,
         'employees': 3564,
         'helped': 1797,
         'got': 45465,
         '2': 31295,
         '-': 246899,
         'item': 3624,
         'jade': 168,
         'chicken': 47061,
         'hengrenade': 5,
         'rice(jerk': 1,
         ')': 123529,
         'ordered': 46768,
         'quesadil

In [49]:

len(counts)

133971

In [50]:
# Vocabulary
vocab2index = {"":0, "UNK":1}
words = ["", "UNK"]
for word in counts:
    vocab2index[word] = len(words)
    words.append(word)

In [51]:
df_reviews['len_text'] = df_reviews['text'].apply(lambda x: len(x.split()))

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [52]:
# WHat is the 99% quantile of  length of the sentence?
df_reviews['len_text'].quantile(0.99)

766.0

In [53]:
# WHat is the 95% quantile of  length of the sentence?
df_reviews['len_text'].quantile(0.95)

494.0

In [54]:
# note that spacy_tok takes a while run it just once
def encode_sentence(sent, vocab2index, N=500, padding_start=True):
    "Encoding a sentence adding padding"
    x = remove_stop_words(spacy_tok(sent))
    enc = np.zeros(N, dtype=np.int32)
    enc1 = np.array([vocab2index.get(w, vocab2index["UNK"]) for w in x])
    l = min(N, len(enc1))
    if padding_start:
        enc[:l] = enc1[:l]
    else:
        enc[N-l:] = enc1[:l]
    return enc, l

In [55]:
l1 = [encode_sentence(sent, vocab2index, N=600, padding_start=False) for sent in df_reviews['text'][:5]]

In [56]:
l1

[(array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,

### Splitting into train and validation sets

In [57]:
from sklearn.model_selection import train_test_split

In [58]:
 X_train, X_valid, y_train, y_valid = train_test_split(df_reviews['text'], df_reviews['fun_bin'], test_size=0.2, random_state=42)

In [59]:
X_train.reset_index(inplace=True, drop=True)
X_valid.reset_index(inplace=True, drop=True)

In [60]:
y_train.reset_index(inplace=True, drop=True)

In [61]:
y_valid.reset_index(inplace=True, drop=True)

In [62]:
X_train[5]

'The reviews are great, but my experience at Stone Neapolitan was not.  My friend was staying in the Grand Wyndham, and we were trying to find somewhere inexpensive for dinner.  Stone Neapolitan does not have much competition on this front.  Most places are lunch only or way overpriced.  So we came here.\n\nWe were the only people in the place, and we both ordered margherita pizzas.  The pizzas looked amazing, but the crust was raw on the insides.  The center was cold and completely raw.  I only ate one slice of pizza, and I took the rest home.  I tried to eat more at one point, but I ended up just throwing the pizza away.  It was not appetizing.\n\nMaybe it was just a bad night, but this pizza did not work for me.'

### Writing a dataset

In [63]:
class YelpDataset(Dataset):
    def __init__(self, df, y, N=500, padding_start=True):
        self.df = df
        self.X = [encode_sentence(sent, vocab2index, N, padding_start) for sent in self.df]
        self.y = y
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        x, s = self.X[idx]
        return x, s, self.y[idx]


In [64]:
train_ds =  YelpDataset(X_train, y_train, padding_start=False)
valid_ds =  YelpDataset(X_valid, y_valid, padding_start=False)

#### Check training and validation datasets for empty sentences

In [65]:
# training
neg=[]
i=0
for x,s,y in train_ds:
    if s <=0:
        neg.append(i)
    i+=1

In [66]:
neg

[118582]

In [67]:
# validation
neg=[]
i=0
for x,s,y in valid_ds:
    if s <=0:
        neg.append(i)
    i+=1

In [68]:
neg

[]

In [69]:
X_train[118582]

'Neither'

In [70]:
train_ds[118582]

(array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [71]:
X_train.shape

(120401,)

In [72]:
X_train.drop([118582], inplace=True)

In [73]:
X_train.shape

(120400,)

In [74]:
X_train.reset_index(drop=True, inplace=True)

In [75]:
y_train.drop([118582], inplace=True)

In [76]:
y_train.shape

(120400,)

In [77]:
y_train.reset_index(drop=True, inplace=True)

In [78]:
train_ds =  YelpDataset(X_train, y_train, padding_start=False)
valid_ds =  YelpDataset(X_valid, y_valid, padding_start=False)

In [79]:
train_ds[118582]

(array([     0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0, 

In [80]:
len(y_valid)

30101

In [81]:
batch_size = 500
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size=batch_size)

In [82]:
np.where(df_reviews.isna())

(array([], dtype=int64), array([], dtype=int64))

In [83]:
class GRU_CNN(torch.nn.Module) :
    def __init__(self, vocab_size, embedding_dim, hidden_dim, out_channels=100) :
        super(GRU_CNN,self).__init__()
        self.hidden_dim = hidden_dim
        self.out_channels = out_channels
        self.dropout = nn.Dropout(0.5)
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.gru = nn.GRU(embedding_dim, hidden_dim, batch_first=True)
        self.conv_3 = nn.Conv1d(in_channels=hidden_dim, out_channels=100, kernel_size=3)
        self.conv_4 = nn.Conv1d(in_channels=hidden_dim, out_channels=100, kernel_size=4)
        self.conv_5 = nn.Conv1d(in_channels=hidden_dim, out_channels=100, kernel_size=5)
        self.linear = nn.Linear(3*out_channels+embedding_dim, 1)        
        
    def forward(self, x, s):
        # sorting
        s, sort_index = torch.sort(s.float(), 0,descending=True) # s is the length of the sentence. Sort these lengths
        s = s.numpy().tolist() # 
        x_gru = x[sort_index]
        x_gru = self.embeddings(x_gru)
        x_gru = self.dropout(x_gru)
        x_pack = pack_padded_sequence(x_gru, s, batch_first=True) # We want LSTM to forget the padding, but in order to apply 
        #ordering mini batches withtin the model
        out_pack, ht = self.gru(x_pack) 
        #out_lstm = self.linear(ht[-1]) # Problem here is that output is not sorted!
        #out_lstm = torch.zeros_like(out_lstm).scatter_(0, sort_index.unsqueeze(1).cuda(), out_lstm)
        # scatter_ is undoing the sorting with the given sorting index
        # kind of sorting back with the original indexing
        x_cnn = self.embeddings(x)
        #print(x_cnn.size())
        x_cnn = x_cnn.transpose(1,2)
        #print(x_cnn.size())
        x3 = F.relu(self.conv_3(x_cnn))
        x4 = F.relu(self.conv_4(x_cnn))
        x5 = F.relu(self.conv_5(x_cnn))
        x3 = nn.MaxPool1d(kernel_size = 398)(x3)
        x4 = nn.MaxPool1d(kernel_size = 397)(x4)
        x5 = nn.MaxPool1d(kernel_size = 396)(x5)
        out_cnn = torch.cat([x3, x4, x5], 2)
        #print(out_cnn.size())
        out_cnn = out_cnn.view(out_cnn.size(0), -1)
        out = torch.cat([out_cnn, ht[-1]],1)
        out = self.dropout(out)
        #print(out.size())
        out = self.linear(out)
        return out 
        

In [84]:
def val_metrics(m, valid_dl):
    model.eval()
    correct = 0
    total = 0
    sum_loss = 0.0
    for x, s, y in valid_dl:
        x = x.long().cuda()
        y = y.float().cuda().unsqueeze(1)
        y_hat = model(x,s)
        loss = F.binary_cross_entropy_with_logits(y_hat, y)
        y_pred = y_hat > 0
        correct += (y_pred.float() == y).float().sum()
        total += y.shape[0]
        sum_loss += loss.item()*y.shape[0]
    return sum_loss/total, correct/total

In [85]:
def train_epocs(model, epochs=10, lr=0.001):
    parameters = filter(lambda p: p.requires_grad, model.parameters())
    optimizer = torch.optim.Adam(parameters, lr=lr)
    for i in range(epochs):
        model.train()
        sum_loss = 0.0
        total = 0
        for x, s, y in train_dl:
            x = x.long().cuda()
            y = y.float().cuda()
            y_pred = model(x, s)
            optimizer.zero_grad()
            loss = F.binary_cross_entropy_with_logits(y_pred, y.unsqueeze(1))
            loss.backward()
            optimizer.step()
            sum_loss += loss.item()*y.shape[0]
            total += y.shape[0]
        val_loss, val_acc = val_metrics(model, valid_dl)
        print("train loss %.3f val loss %.3f and val accuracy %.3f" % (sum_loss/total, val_loss, val_acc))

In [86]:
def save_model(m, p): torch.save(m.state_dict(), p)
    
def load_model(m, p): m.load_state_dict(torch.load(p))

In [87]:
vocab_size = len(words)
print(vocab_size)

133973


In [88]:
model = GRU_CNN(vocab_size, 100,100).cuda()

In [89]:
train_epocs(model, epochs=5, lr=0.1)

train loss 2.474 val loss 2.441 and val accuracy 0.651
train loss 4.701 val loss 11.079 and val accuracy 0.634
train loss 5.772 val loss 6.563 and val accuracy 0.483
train loss 3.808 val loss 9.222 and val accuracy 0.496
train loss 3.644 val loss 42.838 and val accuracy 0.595


The model is not stable, so it's better to lower the learning rate.

In [90]:
model = GRU_CNN(vocab_size, 100,100).cuda()

In [91]:
train_epocs(model, epochs=5, lr=0.01)

train loss 0.601 val loss 0.583 and val accuracy 0.722
train loss 0.578 val loss 0.571 and val accuracy 0.721
train loss 0.545 val loss 0.597 and val accuracy 0.708
train loss 0.506 val loss 0.657 and val accuracy 0.696
train loss 0.476 val loss 0.776 and val accuracy 0.702


In [92]:
train_epocs(model, epochs=5, lr=0.001)

train loss 0.442 val loss 0.951 and val accuracy 0.711
train loss 0.432 val loss 1.022 and val accuracy 0.710
train loss 0.426 val loss 1.109 and val accuracy 0.710
train loss 0.422 val loss 1.191 and val accuracy 0.709
train loss 0.419 val loss 1.242 and val accuracy 0.704


The model seems to learn really fast. Then we will reduce the number of epochs.

In [102]:
model = GRU_CNN(vocab_size, 100,100).cuda()

In [103]:
train_epocs(model, epochs=2, lr=0.01)

train loss 0.599 val loss 0.593 and val accuracy 0.724
train loss 0.573 val loss 0.590 and val accuracy 0.724


In [100]:
save_model(model, '../model_lr0.01_2.pth')

In [101]:
train_epocs(model, epochs=5, lr=0.0001)

train loss 0.519 val loss 0.585 and val accuracy 0.719
train loss 0.512 val loss 0.592 and val accuracy 0.718
train loss 0.509 val loss 0.594 and val accuracy 0.717
train loss 0.506 val loss 0.599 and val accuracy 0.718
train loss 0.504 val loss 0.599 and val accuracy 0.717


In [104]:
train_epocs(model, epochs=3, lr=0.00001)

train loss 0.530 val loss 0.574 and val accuracy 0.721
train loss 0.526 val loss 0.575 and val accuracy 0.721
train loss 0.525 val loss 0.576 and val accuracy 0.721


In [105]:
train_epocs(model, epochs=3, lr=0.00001)

train loss 0.523 val loss 0.578 and val accuracy 0.721
train loss 0.522 val loss 0.578 and val accuracy 0.720
train loss 0.521 val loss 0.579 and val accuracy 0.719
